In [ ]:
%load_ext autoreload
%autoreload 2

import os
from corescpy.class_sc import Omics
import pandas as pd

ddd = "/home/asline01/projects/data/ileal-cd/CD-1"  # cellranger directory

file_path = os.path.join(ddd, "filtered_feature_bc_matrix")
col_sample_id = "orig.ident"
kws_harmony = {"plot_convergence": True,
               "random_state": 1618}
kws_init = {"assay": None, "assay_protein": None, 
            "col_gene_symbols": "gene_symbols", 
            "col_cell_type": "majority_voting", 
            "col_sample_id": None,
            "col_condition": None, 
            "key_control": None, 
            "key_treatment": None}
kws_pp = {"kws_hvg": {"min_mean": 0.0125, "max_mean": 10, 
                      "min_disp": 0.5, "flavor": "cell_ranger"}, 
          "target_sum": 10000.0, "cell_filter_pmt": [0, 15], 
          "cell_filter_ngene": [200, None], "cell_filter_ncounts": [3, 40000], 
          "gene_filter_ncell": [3, None], "regress_out": None, 
          "kws_scale": "log"}
kws_cluster = {"method_cluster": "leiden", "kws_umap": {"min_dist": 0.3}, 
               "kws_neighbors": None,  "resolution": 1,
               "kws_pca": {"n_comps": None, "use_highly_variable": True}, 
               "resolution": 0.5, "kws_cluster": {}}

# Single-Cell Omics Object Initialization

In [4]:
self = Omics(file_path, **kws_init)



<<< INITIALIZING CRISPR CLASS OBJECT >>>

col_gene_symbols="gene_symbols"
col_cell_type="majority_voting"
col_sample_id=None
col_batch=None
col_condition=None
col_num_umis=None
key_control=None
key_treatment=None

<<< LOADING FILE /home/asline01/projects/data/ileal-cd/CD-1/filtered_feature_bc_matrix with sc.read() >>>


ValueError: Reading with filekey '/home/asline01/projects/data/ileal-cd/CD-1/filtered_feature_bc_matrix' failed, the inferred filename PosixPath('/home/asline01/projects/data/ileal-cd/CD-1/filtered_feature_bc_matrix.h5ad') does not exist. If you intended to provide a filename, either use a filename ending on one of the available extensions {'csv', 'data', 'tsv', 'h5ad', 'mtx.gz', 'xlsx', 'tab', 'loom', 'h5', 'anndata', 'txt', 'soft.gz', 'mtx'} or pass the parameter `ext`.

# Preprocessing

Perform normalization, filtering, QC, etc.

In [ ]:
self.preprocess(**kws_pp)

# Clustering

Use this method to perform dimensionality reduction and clustering.

Optionally, by specifying `model_celltypist`, we can also detect cell types using `CellTypist`.

You can provide a CellTypist model name as a string, or to train your own custom model, provide an AnnData object with the training data and, if cell type column names differ across training and main data (in `self`) or if you want to pass additional arguments to `celltypist.train()`, then provide a dictionary of arguments to `kws_train`. (Cell type name can be under `labels` or `col_cell_type` in this dictonary.)

You can see what models are available with this code:
```
import celltypist
celltypist.models.models_description().set_index("model").description.apply(lambda x: print(x))
```

Or it will show you valid models automatically if you feed it an unknown model: `self.cluster(**kws_cluster, model="kate_moss.pkl")``

You can perform this after clustering by leaving `model_celltypist` as None and at any point running `self.annotate_clusters(...)`.

In [ ]:
self.cluster(**kws_cluster)
_ = self.annotate_clusters("COVID19_HumanChallenge_Blood.pkl")

# Plots

Create a variety of plots visualizing gene expression, cell clusters, etc.

If you want to focus on a cell type categorization column other than 

You can often specify splitting/color-coding columns to be different across different plots by specifying a different `col_cell_type` argument in their respective `kws_<PLOT>` arguments (e.g., `kws_umap="predicted_labels"`).

By specifying `color` in `kws_umap`, you can create an additional UMAP colored by a column other than the cell type. If you also specified `cell_types_circle` in the main list of arguments, the UMAP will be colored by `kws_umap["color"]` (or the cell type column, if color isn't specified), and the cell types listed in `cell_types_circle` will be circled on it. 

In [ ]:
clus = list(self.rna.obs["predicted_labels"].sample(2))  # clusters to circle
kgh = list(pd.Series(self.rna.var_names).sample(2))  # gene labels: highlight 
figs = self.plot(genes=16,  # or could specify list of gene names
                 col_cell_type="majority_voting",
                 cell_types_circle=clus,  # list cell types to circle on UMAP
                 kws_gex_violin=dict(scale="area", height=10),
                 genes_highlight=kgh,  # could be genes names to highlight
                 kws_umap=dict(col_cell_type="predicted_labels",
                               color=None))  # for UMAP